In [4]:
!wget "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
!unzip tiny-imagenet-200.zip

Streaming output truncated to the last 5000 lines.
  inflating: tiny-imagenet-200/val/images/val_3963.JPEG  
  inflating: tiny-imagenet-200/val/images/val_7199.JPEG  
  inflating: tiny-imagenet-200/val/images/val_2752.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9687.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9407.JPEG  
  inflating: tiny-imagenet-200/val/images/val_3603.JPEG  
  inflating: tiny-imagenet-200/val/images/val_3412.JPEG  
  inflating: tiny-imagenet-200/val/images/val_6982.JPEG  
  inflating: tiny-imagenet-200/val/images/val_8496.JPEG  
  inflating: tiny-imagenet-200/val/images/val_7332.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9241.JPEG  
  inflating: tiny-imagenet-200/val/images/val_4196.JPEG  
  inflating: tiny-imagenet-200/val/images/val_5980.JPEG  
  inflating: tiny-imagenet-200/val/images/val_6697.JPEG  
  inflating: tiny-imagenet-200/val/images/val_9969.JPEG  
  inflating: tiny-imagenet-200/val/images/val_1598.JPEG  
  inflating: tiny-ima

# Custom dataset


In [0]:
import os
import torch
import torchvision
from torch.utils.data import Dataset

from PIL import Image


def pil_loader(image_path):
    with open(image_path, 'rb') as f:
        return Image.open(f).convert('RGB')


class CustomTrainDataset(Dataset):

    def __init__(self, data_dir, transform=None):
        """
        Args:
            data_dir: the dataset folder. The folder should be structured as:
                label1/image/500 .JPEG images
                label2/image/500 .JPEG images
                ...
                label200/image/500 .JPEG images
        """
        self.data_dir = data_dir
        self.transform = transform
        self.samples = []
        

        #TODO: implement the needed initialization

        self.classes = sorted(os.listdir(data_dir))
        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}

        print(data_dir)
        for _root, _dirs, files in sorted(os.walk(data_dir)):
          for directory in _dirs:
            idx = self.class_to_idx[directory]
            directory_path = os.path.join(data_dir, directory)
            all_images_path = os.path.join(directory_path, 'images')
            print("one of 200 directory paths:", directory_path)
            #directory_path = os.path.join(data_dir, 'images')
            for curr_root, _subdirs, subfiles in sorted(os.walk(all_images_path)):
              #for cur_cur_root, _subsubdir, subsubfile in 
              for image_name in subfiles:
                image_path = os.path.join(curr_root, image_name)
                self.samples.append((image_path, idx))
          break


          

    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        """Don't forget to apply the transform before returning."""
        path, target = self.samples[idx]
        sample = pil_loader(path)
        if self.transform is not None:  
            sample = self.transform(sample)

        return sample, target
        

class CustomValDataset(Dataset):
    """This is the Validation Dataset.

    Use this as a reference to implement you CustomTrainDataset.
    Remember because the training data and validation data are structured differently,
    you shouldn't directly use the code below. You should design it according to the
    training data folder's structure.
    """
    def __init__(self, data_dir, transform=None):
        self.root = os.path.normpath(data_dir)
        self.transform = transform

        classes_file = os.path.join(self.root, '../wnids.txt')

        self.classes = []
        with open(classes_file, 'r') as f:
            self.classes = f.readlines()
            for i in range(len(self.classes)):
                self.classes[i] = self.classes[i].strip()
        self.classes.sort()

        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}

        images_y_file = os.path.join(self.root, 'val_annotations.txt')
        with open(images_y_file, 'r') as f:
            lines = f.readlines()

        self.class_dict = {}
        for line in lines:
            cols = line.split('\t')
            if len(cols) < 2:
                continue
            img_filename = cols[0].strip()
            img_class = cols[1].strip()
            self.class_dict[img_filename] = img_class

        self.samples = []
        images_dir = os.path.join(self.root, 'images')
        for _root, _dirs, files in sorted(os.walk(images_dir)):
            for image_name in files:
                image_path = os.path.join(_root, image_name)
                c = self.class_dict[image_name]
                idx = self.class_to_idx[c]
                self.samples.append((image_path, idx))


    def __getitem__(self, i):
        path, target = self.samples[i]
        sample = pil_loader(path)
        if self.transform is not None:
            sample = self.transform(sample)

        return sample, target

    def __len__(self):
        return len(self.samples)

# Custom Alexnet

In [0]:
import torch
import torch.nn as nn

class CustomAlexnet(nn.Module):
    """Alexnet implementation for comp 590.
    
    Remember, you are required to implement alexnet using
    nn.Conv2d instead of nn.Linear.
    Failing to do that will lead to 0 points for this task.
    """
    def __init__(self, num_classes=200):
        super(CustomAlexnet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=8, stride=2, padding=2),
            nn.ReLU(inplace=True),
            #nn.MaxPool2d(kernel_size=3, stride=2),
            nn.MaxPool2d(kernel_size=3, stride=1),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Conv2d(256, 4096, kernel_size=6),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Conv2d(4096, 4096, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(4096, 200, kernel_size=1),
        )
        

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        x = torch.flatten(x, 1)
        return x

# Helper functions

In [0]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

# Train and validation functions

In [0]:
import os
import random
import shutil
import time
import warnings

def train(train_loader, model, criterion, optimizer, epoch, dev):

    print_frequency = 10
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        images = images.to(dev)
        target = target.to(dev)

        # compute output
        output = model(images)
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_frequency == 0:
            progress.display(i)
        
    return losses.avg


def validate(val_loader, model, criterion, dev):

    print_frequency = 10
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            images = images.to(dev)
            target = target.to(dev)            

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_frequency == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg, top5.avg, losses.avg


# Main block


In [10]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torchvision.transforms as transforms
from torchsummary import summary


if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"


model = CustomAlexnet(200)
datadir = './tiny-imagenet-200'

model.to(dev)

# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss().to(dev)

#lr = 0.01
lr = 0.005

momentum = 0.9
weight_decay = 1e-4
optimizer = torch.optim.SGD(model.parameters(), lr=lr,
                            momentum=momentum,
                            weight_decay=weight_decay)

# Data loading code
traindir = os.path.join(datadir, 'train')
valdir = os.path.join(datadir, 'val')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])

print('=> using CustomTrainDataset')
train_dataset = CustomTrainDataset(
    traindir,
    transforms.Compose([
        # transforms.RandomResizedCrop(64),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
print('=> using CustomValDataset')
val_dataset = CustomValDataset(
    valdir,
    transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

# The following 3 lines are for verification of your model structure
# They should output exactly the same lines as the picture in the instruction
print('For 3x64x64 images, the model will generate outputs in the following shapes:')
summary(model, (3, 64, 64))
assert train_dataset.classes == val_dataset.classes

# Modify batch size to see the difference
batch_size = 64
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=4, pin_memory=True)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False,
    num_workers=4, pin_memory=True)


best_acc1=0
#epochs=100
epochs=20
for epoch in range(epochs):

    # train for one epoch
    train_loss = train(train_loader, model, criterion, optimizer, epoch, dev)

    # evaluate on validation set
    acc1, acc5, val_loss = validate(val_loader, model, criterion, dev)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)

    # Don't forget to include model_best.pth.tar in your report
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)




=> using CustomTrainDataset
./tiny-imagenet-200/train
one of 200 directory paths: ./tiny-imagenet-200/train/n02123045
one of 200 directory paths: ./tiny-imagenet-200/train/n07749582
one of 200 directory paths: ./tiny-imagenet-200/train/n03662601
one of 200 directory paths: ./tiny-imagenet-200/train/n01641577
one of 200 directory paths: ./tiny-imagenet-200/train/n04275548
one of 200 directory paths: ./tiny-imagenet-200/train/n02843684
one of 200 directory paths: ./tiny-imagenet-200/train/n01984695
one of 200 directory paths: ./tiny-imagenet-200/train/n04099969
one of 200 directory paths: ./tiny-imagenet-200/train/n02486410
one of 200 directory paths: ./tiny-imagenet-200/train/n01698640
one of 200 directory paths: ./tiny-imagenet-200/train/n07720875
one of 200 directory paths: ./tiny-imagenet-200/train/n02395406
one of 200 directory paths: ./tiny-imagenet-200/train/n02106662
one of 200 directory paths: ./tiny-imagenet-200/train/n03976657
one of 200 directory paths: ./tiny-imagenet-200/tr

In [12]:
import torch
checkpoint = torch.load('checkpoint.pth.tar')
start_epoch = checkpoint['epoch']
best_acc1 = checkpoint['best_acc1']
state_dict = checkpoint['state_dict']
optimizer = checkpoint['optimizer']
#net = checkpoint['model']
print(start_epoch, best_acc1)
#print(state_dict)
print(optimizer)

20 tensor(39.8000, device='cuda:0')
{'state': {139779386523936: {'momentum_buffer': tensor([[[[ 8.5112e-02,  8.9892e-02,  8.9103e-02,  ...,  7.4497e-02,
            6.9597e-02,  5.9090e-02],
          [ 5.9741e-02,  9.3384e-02,  1.1785e-01,  ...,  6.8584e-02,
            6.6072e-02,  5.9641e-02],
          [ 5.8081e-02,  8.0381e-02,  9.0374e-02,  ...,  5.9592e-02,
            6.9840e-02,  4.9030e-02],
          ...,
          [ 2.7192e-02,  5.0878e-02,  3.4015e-02,  ...,  1.2999e-02,
            2.7559e-02,  2.9022e-02],
          [ 6.4870e-03,  2.5114e-02,  2.0687e-02,  ...,  1.4047e-02,
            2.1864e-02,  1.7097e-02],
          [-8.3269e-03, -7.9116e-03, -5.2215e-03,  ..., -1.1458e-02,
           -1.4806e-02,  7.1173e-03]],

         [[ 6.4703e-02,  6.9178e-02,  6.8002e-02,  ...,  6.3425e-02,
            5.7576e-02,  5.0683e-02],
          [ 3.8815e-02,  7.2604e-02,  9.9097e-02,  ...,  5.8725e-02,
            5.6861e-02,  5.2949e-02],
          [ 3.4961e-02,  5.7767e-02,  7.147